In [2]:
'''
cutFinder
by Leo d'Espaux <leodespaux@gmail.com>
with help from William Zhuang, Kai Li

Finds cut sites within a user-input sequence, then checks those candidates against
chromosome files discarding sequences found more than once. 
Output is a list of candidate 20mer gRNAs in your target sequence.

Details:
*For now only looks in the sense strand of input, but checks both sense and antisense on the genome.
*The program lists these sequences as it finds them. 
*Usually, we get a cut site every ~25nt.
*Note your target can be a sequence that's not originally in the genome.

Current as of 8/30/16

'''

# import libraries
from Bio.Seq import Seq
from Bio import SeqIO
from Bio import Entrez
from pprint import pprint




def findCutSites():
    print("Hi! Welcome to cutFinder. \nFor questions or comments email leodespaux@gmail.com\n\n")
    print("If you're using Python2, use quotes in your answers.")
    print("I will only look at the sense strand of your target.\n\n")


    print("I have the following genomes: S288c, CENpk, PO1f, CLIB, SAKL.")
    species=input("Which one do you want? e.g., S288c: ")
    print("")
    print("Hold tight while I fetch that genome...\n")
    
    if species=="PO1f":
        allRecords=[SeqIO.read("Yl_PO1f_A.fasta", "fasta"), SeqIO.read("Yl_PO1f_B.fasta", "fasta"),
                    SeqIO.read("Yl_PO1f_C.fasta", "fasta"), SeqIO.read("Yl_PO1f_D.fasta", "fasta"),
                    SeqIO.read("Yl_PO1f_E.fasta", "fasta"), SeqIO.read("Yl_PO1f_F.fasta", "fasta")]

    elif species=="CLIB":
        allRecords=[SeqIO.read("Yali0A_contig.fasta", "fasta"), SeqIO.read("Yali0B_contig.fasta", "fasta"),
            SeqIO.read("Yali0C_contig.fasta", "fasta"), SeqIO.read("Yali0D_contig.fasta", "fasta"),
            SeqIO.read("Yali0E_contig.fasta", "fasta"), SeqIO.read("Yali0F_contig.fasta", "fasta")]

    elif species=="SAKL":
        allRecords=[SeqIO.read("chromosomeA.fasta", "fasta"), SeqIO.read("chromosomeB.fasta", "fasta"),
                    SeqIO.read("chromosomeC.fasta", "fasta"), SeqIO.read("chromosomeD.fasta", "fasta"),
                    SeqIO.read("chromosomeE.fasta", "fasta"), SeqIO.read("chromosomeF.fasta", "fasta")]
        
    elif species=="S288c":
        allRecords=[SeqIO.read("Scer01.fasta", "fasta"), SeqIO.read("Scer02.fasta", "fasta"),
                    SeqIO.read("Scer03.fasta", "fasta"), SeqIO.read("Scer04.fasta", "fasta"),
                    SeqIO.read("Scer05.fasta", "fasta"), SeqIO.read("Scer06.fasta", "fasta"),
                    SeqIO.read("Scer07.fasta", "fasta"), SeqIO.read("Scer08.fasta", "fasta"),
                    SeqIO.read("Scer09.fasta", "fasta"), SeqIO.read("Scer10.fasta", "fasta"),
                    SeqIO.read("Scer11.fasta", "fasta"), SeqIO.read("Scer12.fasta", "fasta"),
                    SeqIO.read("Scer13.fasta", "fasta"), SeqIO.read("Scer14.fasta", "fasta"),
                    SeqIO.read("Scer15.fasta", "fasta"), SeqIO.read("Scer16.fasta", "fasta")]
        
    elif species=="CENpk":
        allRecords=[SeqIO.read("CENPK113-7D_CH1.fasta", "fasta"), SeqIO.read("CENPK113-7D_CH2.fasta", "fasta"),
                    SeqIO.read("CENPK113-7D_CH3.fasta", "fasta"), SeqIO.read("CENPK113-7D_CH4.fasta", "fasta"),
                    SeqIO.read("CENPK113-7D_CH5.fasta", "fasta"), SeqIO.read("CENPK113-7D_CH6.fasta", "fasta"),
                    SeqIO.read("CENPK113-7D_CH7.fasta", "fasta"), SeqIO.read("CENPK113-7D_CH8.fasta", "fasta"),
                    SeqIO.read("CENPK113-7D_CH9.fasta", "fasta"), SeqIO.read("CENPK113-7D_CH10.fasta", "fasta"),
                    SeqIO.read("CENPK113-7D_CH11.fasta", "fasta"), SeqIO.read("CENPK113-7D_CH12.fasta", "fasta"),
                    SeqIO.read("CENPK113-7D_CH13.fasta", "fasta"), SeqIO.read("CENPK113-7D_CH14.fasta", "fasta"),
                    SeqIO.read("CENPK113-7D_CH15.fasta", "fasta"), SeqIO.read("CENPK113-7D_CH16.fasta", "fasta"),
                    SeqIO.read("CENPK113-7D_mitochondria.fasta", "fasta")]
        
    print("I found the following genomic records: ")
    for record in allRecords: # a record is a chromosome file
        print( record.description)
    print("")

    
    
    # ask for target and see if it's present and unique
    print("OK, what sequence do you want to find cut sites in: ")
    targetSeq=Seq(input().replace(" ","").upper()) #get rid of spaces and make all caps
    
 
    targetCount=checkUnique(allRecords, targetSeq)
    print("\nFound "+str(targetCount)+" instances of your target sequence in this genome.")
    print("Now I'll find cut sites. I'll print them out as I find them so you know I'm working.\n")
    
    #some sensible checks
    #if targetCount==0:
     #   print("Is this a heterologous gene? I'll still find cut sites.")
        
    #elif targetCount>1:
      #  print("There are multiple instances of your target sequence in this genome. I'll still find cut sites.")
    
    #elif targetCount==1:
      #  print("Your target sequence is found in the genome once.")
    

    # make an empty list of cut sites
    cutters=[]    
    
    # let's look through our locus to find cut sites. We end at len-24, since we are looking thru 23-nts,  
    # and because python includes the first index but not the last one
    
    
    for i in range(len(targetSeq)-24):

        # sub is an array containing all 23mers
        sub=targetSeq[i:i+23]
        uniqueCount=0
        degenerateCount=0
        NAGcount=0 
        # first, cut sequences are N20NGG, cannot contain TTTTTT since that terminates PolIII transcription
        if sub[21]=="G" and sub[22]=='G' and not('TTTTTT' in sub):
           
            # First, we're going to look to find instances of N20 followed by any NGG. If there's more than once, then the cutter is NOT unique and we discard.
        
            if (checkUnique(allRecords,sub[1:20]+"AGG")+checkUnique(allRecords,sub[1:20]+"TGG")+checkUnique(allRecords,sub[1:20]+"CGG")+checkUnique(allRecords,sub[1:20]+"GGG")) > 1:
                break # If you find more than one instance of candidate N20 followed by any NGG, discard
            
            else:
                #degenerate count is the count of all instances of the seed sequence. Less the better.
                degenerateCount=checkUnique(allRecords,sub[8:20]+"AGG")+checkUnique(allRecords,sub[8:20]+"TGG")+checkUnique(allRecords,sub[8:20]+"CGG")+checkUnique(allRecords,sub[8:20]+"GGG") 
                
                #NAG count is the count of instances of the seed sequence followed by NAG, which can have off target effects. Less is better.
                NAGcount=checkUnique(allRecords,sub[8:20]+"AAG")+checkUnique(allRecords,sub[8:20]+"TAG")+checkUnique(allRecords,sub[8:20]+"CAG")+checkUnique(allRecords,sub[8:20]+"GAG")
                
                #Then append and print.
                cutters.append([str(sub[0:20]), degenerateCount, NAGcount])
                print(sub[0:20])
                
    if not cutters:
        print("\nSorry didn't find any suitable cut sites. Consider starting again with the rcomplement of your target.")
    else:
        print("\nThese are the cut sites I found, and their degenerate and NAG counts.")  
        print("The lower counts are better. If you want, also try the rcomplement of your target as input.\n\n")
        pprint(cutters)

    #print("degenerate, NAG counts: ")
    #print(degenerateCount)
    #print(NAGcount)

 

    
            
            
            
def checkUnique(searchRecords, string):
# Here we take in a string sequence and see if it's unique in the records (chromosomes)
# defined earlier. If unique, returns 1; if not found, 0; and if found more than once, 2.
        
        nfound=0
        
        for record in searchRecords:
            nfound=nfound+record.seq.count(str(string))
            if nfound>1:
                return 2
                break
            # note that we should look in the other strand, too    
            nfound=nfound+record.seq.reverse_complement().count(str(string))
            if nfound>1:
                return 2
                break
            
        if nfound == 0:
            return 0
            
        if nfound == 1:
            return 1
            
            
findCutSites()
                    
                        
                    



Hi! Welcome to cutFinder. 
For questions or comments email leodespaux@gmail.com


If you're using Python2, use quotes in your answers.
I will only look at the sense strand of your target.


I have the following genomes: S288c, CENpk, PO1f, CLIB, SAKL.
Which one do you want? e.g., S288c: S288c

Hold tight while I fetch that genome...

I found the following genomic records: 
tpg|BK006935.2| [organism=Saccharomyces cerevisiae S288c] [strain=S288c] [moltype=genomic] [chromosome=I] [note=R64-1-1]
tpg|BK006936.2| [organism=Saccharomyces cerevisiae S288c] [strain=S288c] [moltype=genomic] [chromosome=II] [note=R64-1-1]
tpg|BK006937.2| [organism=Saccharomyces cerevisiae S288c] [strain=S288c] [moltype=genomic] [chromosome=III] [note=R64-1-1]
tpg|BK006938.2| [organism=Saccharomyces cerevisiae S288c] [strain=S288c] [moltype=genomic] [chromosome=IV] [note=R64-1-1]
tpg|BK006939.2| [organism=Saccharomyces cerevisiae S288c] [strain=S288c] [moltype=genomic] [chromosome=V] [note=R64-1-1]
tpg|BK006940.2|